In [1]:
# CIFAR-10を読み込んで標準化を含めた前処理を行う
import numpy as np
from keras.datasets import cifar10
from keras.utils import np_utils

# CIFAR-10データを読み込む
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# 画像データをfloat32(浮動小数点数)型に変換
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# 訓練用とテスト用の画像データを標準化する
# 4次元テンソルのすべての軸方向に対して平均、標準偏差を求めるので
# axis=(0,1,2,3)は省略してもよい
mean = np.mean(x_train, axis=(0,1,2,3))
std = np.std(x_train, axis=(0,1,2,3))
# 標準化する際に標準偏差に極小値を加える
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

# 分類先のクラスの数
num_classes = 10
# 正解ラベルをワンホット表現に変換
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.optimizers import RMSprop

# ハイパーパラメーターの値を設定
weight_decay = 1e-4

# CNNを構築
model = Sequential()

# (第1層)畳み込み層1 正則化を行う
model.add(
    Conv2D(
        filters=32,                    # フィルターの数は32
        kernel_size=(3,3),             # 3×3のフィルターを使用
        input_shape=x_train.shape[1:], # 入力データの形状
        padding='same',                # ゼロパディングを行う 
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'              # 活性化関数はReLU
        ))
# 正規化
model.add(BatchNormalization())

# (第2層)畳み込み層2　正則化を行う
model.add(
    Conv2D(filters=32,                 # フィルターの数は32
           kernel_size=(3,3),          # 3×3のフィルターを使用
           padding='same',             # ゼロパディングを行う 
           kernel_regularizer=regularizers.l2(weight_decay),
           activation='relu'           # 活性化関数はReLU
           ))
# 正規化
model.add(BatchNormalization())

# (第3層)プーリング層1：ウィンドウサイズは2×2
model.add(
    MaxPooling2D(pool_size=(2,2)))
# ドロップアウト1：ドロップアウトは20％
model.add(Dropout(0.2))
 
# (第4層)畳み込み層3　正則化を行う
model.add(
    Conv2D(filters=64,                 # フィルターの数は64
           kernel_size=(3,3),          # 3×3のフィルターを使用
           padding='same',             # ゼロパディングを行う 
           kernel_regularizer=regularizers.l2(weight_decay),
           activation='relu'           # 活性化関数はReLU
           ))

# 正規化
model.add(BatchNormalization())

# (第5層)畳み込み層3　正則化を行う
model.add(
    Conv2D(filters=32,                 # フィルターの数は32
           kernel_size=(3,3),          # 3×3のフィルターを使用
           padding='same',             # ゼロパディングを行う 
           kernel_regularizer=regularizers.l2(weight_decay),
           activation='relu'           # 活性化関数はReLU
           ))
# 正規化
model.add(BatchNormalization())

# (第6層)プーリング層2：ウィンドウサイズは2×2
model.add(
    MaxPooling2D(pool_size=(2,2)))
# ドロップアウト2：ドロップアウトは30％
model.add(Dropout(0.3))
 
# (第6層)畳み込み層4　正則化を行う
model.add(
    Conv2D(filters=128,                # フィルターの数は128
           kernel_size=(3,3),          # 3×3のフィルターを使用
           padding='same',             # ゼロパディングを行う 
           kernel_regularizer=regularizers.l2(weight_decay),
           activation='relu'           # 活性化関数はReLU
          ))
# 正規化
model.add(BatchNormalization())

# (第7層)畳み込み層5　正則化を行う
model.add(
    Conv2D(filters=128,                # フィルターの数は128
           kernel_size=(3,3),          # 3×3のフィルターを使用
           padding='same',             # ゼロパディングを行う 
           kernel_regularizer=regularizers.l2(weight_decay),
           activation='relu'           # 活性化関数はReLU
          ))
# 正規化
model.add(BatchNormalization())

# (第8層)プーリング層3：ウィンドウサイズは2×2
model.add(
    MaxPooling2D(pool_size=(2,2)))
# ドロップアウト2：ドロップアウトは40％
model.add(Dropout(0.4))
 
# Flatten層　4階テンソルから2階テンソルに変換
model.add(Flatten())

# （第9層）出力層
model.add(
    Dense(num_classes,           # 出力層のニューロン数はnum_classes
          activation='softmax')) # 活性化関数はソフトマックス

# Sequentialオブジェクトのコンパイル
model.compile(
    loss='categorical_crossentropy', # 損失の基準は交差エントロピー誤差
    # 最適化アルゴリズムにCNNと相性の良いRMSpropアルゴリズムを使用
    optimizer=RMSprop(lr=0.001, decay=1e-6),
    metrics=['accuracy'] # 学習評価として正解率を指定
)

In [5]:
# 拡張データを使って学習を行う
from keras.preprocessing.image import ImageDataGenerator

# 学習率をスケジューリングする
def lr_schedule(epoch):
    lrate = 0.001      # 1～75回の学習率
    if epoch > 75:
        lrate = 0.0005 # 76～100回の学習率
    elif epoch > 100:
        lrate = 0.0003 # 101回以降の学習率
    return lrate
 
# データ拡張
datagen = ImageDataGenerator(
    rotation_range=15,      # 15度の範囲でランダムに回転させる
    width_shift_range=0.1,  # 横サイズの0.1の割合でランダムに水平移動
    height_shift_range=0.1, # 縦サイズの0.1の割合でランダムに垂直移動
    horizontal_flip=True,   # 水平方向にランダムに反転、左右の入れ替え
    zoom_range=0.2,         # ランダムに拡大
    )
 
# ミニバッチのサイズ
batch_size = 64
# 学習回数
epochs = 130

# 学習を行う
history = model.fit_generator(
    # 拡張データをミニバッチの数だけ生成
    datagen.flow(x_train,
                 y_train,
                 batch_size=batch_size),
    # 1回の学習におけるステップ数
    # 画像の枚数をミニバッチのサイズで割った整数値
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs, # 学習回数
    verbose=1,     # 学習の進捗状況を出力する
    # テストデータ
    validation_data=(x_test,y_test),
    # 学習率のスケジューラーとしてlr_schedule()を呼ぶ
    callbacks=[LearningRateScheduler(lr_schedule)]
)

Epoch 1/130
781/781 [==============================] - 691s 885ms/step - loss: 1.9950 - acc: 0.4032 - val_loss: 1.5210 - val_acc: 0.5086
Epoch 2/130
781/781 [==============================] - 630s 806ms/step - loss: 1.5549 - acc: 0.5315 - val_loss: 1.3740 - val_acc: 0.5600
Epoch 3/130
781/781 [==============================] - 621s 795ms/step - loss: 1.3895 - acc: 0.5886 - val_loss: 1.1134 - val_acc: 0.6655
Epoch 4/130
781/781 [==============================] - 628s 805ms/step - loss: 1.3113 - acc: 0.6225 - val_loss: 1.1123 - val_acc: 0.6648
Epoch 5/130
781/781 [==============================] - 619s 792ms/step - loss: 1.2365 - acc: 0.6460 - val_loss: 1.0438 - val_acc: 0.6972
Epoch 6/130
781/781 [==============================] - 616s 789ms/step - loss: 1.1793 - acc: 0.6677 - val_loss: 1.0105 - val_acc: 0.7125
Epoch 7/130
781/781 [==============================] - 619s 792ms/step - loss: 1.1447 - acc: 0.6814 - val_loss: 0.9913 - val_acc: 0.7051
Epoch 8/130
781/781 [====================

781/781 [==============================] - 638s 817ms/step - loss: 0.6799 - acc: 0.8098 - val_loss: 0.6837 - val_acc: 0.8271
Epoch 61/130
781/781 [==============================] - 637s 816ms/step - loss: 0.6835 - acc: 0.8115 - val_loss: 0.6951 - val_acc: 0.8182
Epoch 62/130
781/781 [==============================] - 635s 814ms/step - loss: 0.6781 - acc: 0.8119 - val_loss: 0.6249 - val_acc: 0.8412
Epoch 63/130
781/781 [==============================] - 635s 814ms/step - loss: 0.6793 - acc: 0.8116 - val_loss: 0.6867 - val_acc: 0.8192
Epoch 64/130
781/781 [==============================] - 644s 825ms/step - loss: 0.6772 - acc: 0.8147 - val_loss: 0.7150 - val_acc: 0.8130
Epoch 65/130
781/781 [==============================] - 640s 820ms/step - loss: 0.6677 - acc: 0.8144 - val_loss: 0.6081 - val_acc: 0.8458
Epoch 66/130
781/781 [==============================] - 640s 820ms/step - loss: 0.6759 - acc: 0.8129 - val_loss: 0.6173 - val_acc: 0.8435
Epoch 67/130
781/781 [=========================

781/781 [==============================] - 682s 874ms/step - loss: 0.5349 - acc: 0.8490 - val_loss: 0.5264 - val_acc: 0.8647
Epoch 120/130
781/781 [==============================] - 706s 904ms/step - loss: 0.5334 - acc: 0.8508 - val_loss: 0.5195 - val_acc: 0.8634
Epoch 121/130
781/781 [==============================] - 710s 908ms/step - loss: 0.5317 - acc: 0.8484 - val_loss: 0.4983 - val_acc: 0.8731
Epoch 122/130
781/781 [==============================] - 678s 869ms/step - loss: 0.5337 - acc: 0.8506 - val_loss: 0.5344 - val_acc: 0.8622
Epoch 123/130
781/781 [==============================] - 680s 871ms/step - loss: 0.5269 - acc: 0.8528 - val_loss: 0.5081 - val_acc: 0.8716
Epoch 124/130
781/781 [==============================] - 684s 876ms/step - loss: 0.5312 - acc: 0.8498 - val_loss: 0.5224 - val_acc: 0.8629
Epoch 125/130
781/781 [==============================] - 681s 872ms/step - loss: 0.5325 - acc: 0.8487 - val_loss: 0.5062 - val_acc: 0.8714
Epoch 126/130
781/781 [==================

In [ ]:
# 学習回数ごとの精度と損失の変化をグラフにする

%matplotlib inline
import matplotlib.pyplot as plt

def plot_history(history):
    # 精度の履歴をプロット
    plt.plot(history.history['acc'],"-",label="accuracy")
    plt.plot(history.history['val_acc'],"-",label="val_acc")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.show()

    # 損失の履歴をプロット
    plt.plot(history.history['loss'],"-",label="loss",)
    plt.plot(history.history['val_loss'],"-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='upper right')
    plt.show()
# modelに学習させた時の変化の様子をplot
plot_history(history)